In [1]:
import pandas as pd
import numpy as np 

train = pd.read_csv('kaggle_data/train.csv')
test = pd.read_csv('kaggle_data/test.csv')

In [2]:
train.head(2)

,rating,movieId,userId
0,3.0,2158,10953099004185006703
1,3.5,12959,10953099004185006703


In [3]:
test.head(2)

,movieId,userId
0,939,3128775406573976678
1,19,6058877902987009152


In [4]:
test['rating']= np.zeros(test.shape[0],dtype=np.float32)
df = train.append(test)

In [5]:
train.shape

(19014573, 3)

In [6]:
df.head(2)

,movieId,rating,userId
0,2158,3.0,10953099004185006703
1,12959,3.5,10953099004185006703


In [7]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix, csr_matrix, csc_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

%matplotlib inline

In [8]:
index_to_item_id_mapping = [x for x in df["movieId"].sort_values().unique()]
item_id_to_index_mapping = {}
for index, value in enumerate(index_to_item_id_mapping, start=1):
    item_id_to_index_mapping[int(value)] = index

In [9]:
df = pd.DataFrame.from_records(
    {"userId": row.userId, "movieId": item_id_to_index_mapping[int(row.movieId)], "rating": row.rating}
    for row in df.itertuples())

In [10]:
!vw --version

8.5.0


In [11]:
train = df[["userId", "movieId", "rating"]].iloc[:19014573]
test = df[["userId", "movieId", "rating"]].iloc[19014573:]

In [12]:
train.to_csv("ml20m_train.csv", header=False, index=False)
test.to_csv("ml20m_test.csv", header=False, index=False)

In [15]:
!awk -F"," '{printf "%f |u %d |i %d\n", $3,$1,$2}' < ml20m_train.csv | \
  vw /dev/stdin -b 18 -q ui --rank 30 --l2 0.001 --learning_rate 0.015 \
    --passes 5 --decay_learning_rate 0.97 --power_t 0 \
    --loss_function squared -f movielens2.reg --cache_file movielens2.cache

creating quadratic features for pairs: ui 
using l2 regularization = 0.001
final_regressor = movielens.reg
Num weight bits = 18
learning rate = 0.015
initial_t = 1
power_t = 0
decay_learning_rate = 0.97
creating cache_file = movielens.cache
Reading datafile = /dev/stdin
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
8.221048 8.221048            1            1.0   3.0000   0.1328       63
7.890895 7.560741            2            2.0   3.0000   0.2503       63
9.189867 10.488839            4            4.0   4.0000   0.3303       63
8.105761 7.021654            8            8.0   4.0000   0.5381       63
8.488322 8.870884           16           16.0   2.0000   0.6615       63
5.895707 3.303092           32           32.0   4.0000   1.3653       63
4.410877 2.926048           64           64.0   3.0000   2.0113       63
3.192592 1.974306          128          128.0   4.0000 

In [45]:
!awk -F"," '{printf "%f |u %d |i %d\n", $3,$1,$2}' < ml20m_test.csv | \
  vw /dev/stdin -i movielens2.reg -t -r pred2.txt

creating quadratic features for pairs: ui 
only testing
raw predictions = pred2.txt
Num weight bits = 20
learning rate = 10
initial_t = 1
power_t = 0.5
using no cache
Reading datafile = /dev/stdin
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
17.262957 17.262957            1            1.0   0.0000   4.1549       63
11.072399 4.881842            2            2.0   0.0000   2.2095       63
11.602318 12.132237            4            4.0   0.0000   3.2041       63
12.328582 13.054847            8            8.0   0.0000   4.0968       63
11.783622 11.238662           16           16.0   0.0000   3.7949       63
11.048606 10.313590           32           32.0   0.0000   3.3668       63
11.467353 11.886100           64           64.0   0.0000   3.5639       63
11.656664 11.845975          128          128.0   0.0000   3.4260       63
11.564241 11.471818          256          

In [14]:
predicted_data = pd.read_csv('pred.txt',names=['rating'],index_col=None)
predicted_data2 = pd.read_csv('pred2.txt',names=['rating'],index_col=None)

In [15]:
predicted = predicted_data['rating'].values
predicted2 = predicted_data2['rating'].values

In [16]:
predicted.shape, test.shape

NameError: name 'test' is not defined

In [17]:
baseline = pd.read_csv('kaggle_data/baseline.csv')

In [18]:
# ans = baseline['rating'].values * 0.6 + predicted * 0.4
ans = 0.5 * (predicted + predicted2)

In [20]:
baseline['rating'] = ans
baseline.to_csv('ans_clear_12.csv',index=False)

1.0